In [104]:
import numpy as np
import pandas as pd
import glob
from joblib import Parallel, delayed
from tqdm import tqdm

In [105]:
application = pd.read_csv('./src/application_train.csv')
# application_test = pd.read_csv('./src/application_test.csv')
# application_test doesnt contain target column, it is useless to us because even if we develop a model, we still dont know its accuracy on the training set. 

In [106]:
bureau = pd.read_csv('./src/bureau.csv')

In [107]:
bureau_balance = pd.read_csv('./src/bureau_balance.csv')

In [108]:
credit_card_balance = pd.read_csv('./src/credit_card_balance.csv')

In [109]:
installments_payments = pd.read_csv('./src/installments_payments.csv')

In [110]:
POS_CASH_balance = pd.read_csv('./src/POS_CASH_balance.csv')
previous_application = pd.read_csv('./src/previous_application.csv')

In [111]:
home_credit_columns_description = pd.read_csv('./src/HomeCredit_columns_description.csv', encoding='iso-8859-1',
                                              index_col=0)

In [112]:
# sample_submission = pd.read_csv('./src/sample_submission.csv')

# Data preprocessing

In [113]:
application.describe()

,SK_ID_CURR,TARGET,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,REGION_POPULATION_RELATIVE,DAYS_BIRTH,DAYS_EMPLOYED,...,FLAG_DOCUMENT_18,FLAG_DOCUMENT_19,FLAG_DOCUMENT_20,FLAG_DOCUMENT_21,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_YEAR
count,307511.000000,307511.000000,307511.000000,3.075110e+05,3.075110e+05,307499.000000,3.072330e+05,307511.000000,307511.000000,307511.000000,...,307511.000000,307511.000000,307511.000000,307511.000000,265992.000000,265992.000000,265992.000000,265992.000000,265992.000000,265992.000000
mean,278180.518577,0.080729,0.417052,1.687979e+05,5.990260e+05,27108.573909,5.383962e+05,0.020868,-16036.995067,63815.045904,...,0.008130,0.000595,0.000507,0.000335,0.006402,0.007000,0.034362,0.267395,0.265474,1.899974
std,102790.175348,0.272419,0.722121,2.371231e+05,4.024908e+05,14493.737315,3.694465e+05,0.013831,4363.988632,141275.766519,...,0.089798,0.024387,0.022518,0.018299,0.083849,0.110757,0.204685,0.916002,0.794056,1.869295
min,100002.000000,0.000000,0.000000,2.565000e+04,4.500000e+04,1615.500000,4.050000e+04,0.000290,-25229.000000,-17912.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,189145.500000,0.000000,0.000000,1.125000e+05,2.700000e+05,16524.000000,2.385000e+05,0.010006,-19682.000000,-2760.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,278202.000000,0.000000,0.000000,1.471500e+05,5.135310e+05,24903.000000,4.500000e+05,0.018850,-15750.000000,-1213.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000
75%,367142.500000,0.000000,1.000000,2.025000e+05,8.086500e+05,34596.000000,6.795000e+05,0.028663,-12413.000000,-289.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,3.000000
max,456255.000000,1.000000,19.000000,1.170000e+08,4.050000e+06,258025.500000,4.050000e+06,0.072508,-7489.000000,365243.000000,...,1.000000,1.000000,1.000000,1.000000,4.000000,9.000000,8.000000,27.000000,261.000000,25.000000


In [114]:
# application.loc[:,['SK_ID_CURR','SK_ID_BUREAU','SK_ID_PREV']]

In [115]:
home_credit_columns_description

,Table,Row,Description,Special
1,application_{train|test}.csv,SK_ID_CURR,ID of loan in our sample,NaN
2,application_{train|test}.csv,TARGET,Target variable (1 - client with payment diffi...,NaN
5,application_{train|test}.csv,NAME_CONTRACT_TYPE,Identification if loan is cash or revolving,NaN
6,application_{train|test}.csv,CODE_GENDER,Gender of the client,NaN
7,application_{train|test}.csv,FLAG_OWN_CAR,Flag if the client owns a car,NaN
...,...,...,...,...
217,installments_payments.csv,NUM_INSTALMENT_NUMBER,On which installment we observe payment,NaN
218,installments_payments.csv,DAYS_INSTALMENT,When the installment of previous credit was su...,time only relative to the application
219,installments_payments.csv,DAYS_ENTRY_PAYMENT,When was the installments of previous credit p...,time only relative to the application
220,installments_payments.csv,AMT_INSTALMENT,What was the prescribed installment amount of ...,NaN


the SK_ID_CURR is the unique id for each client

In [116]:
len(application)

307511

In [117]:
len(bureau)

1716428

In [118]:
len(bureau_balance)

27299925

In [119]:
len(credit_card_balance)

3840312

In [120]:
len(installments_payments)

13605401

In [121]:
len(POS_CASH_balance)

10001358

In [122]:
len(previous_application)

1670214

## merge the bureau data with bureau_balance data

In [123]:
status_count = bureau_balance.groupby('SK_ID_BUREAU')['STATUS'].value_counts().unstack().fillna(0)
status_count.columns = ['BUREAU_BALANCE_' + str(col_name) for col_name in status_count.columns]
status_count.reset_index(inplace=True)
month_count = bureau_balance.groupby('SK_ID_BUREAU')['MONTHS_BALANCE'].min().abs().rename('BUREAU_BALANCE_MONTH_COUNT')
status_count = status_count.merge(month_count, how='left', on='SK_ID_BUREAU')  # add 0.001 to avoid divide by 0

In [124]:
# scale the status_count by month_count
for col in status_count.columns[1:-1]:
    status_count[col] = status_count[col] / (status_count['BUREAU_BALANCE_MONTH_COUNT'] + 0.001)

In [125]:
status_count.columns

Index(['SK_ID_BUREAU', 'BUREAU_BALANCE_0', 'BUREAU_BALANCE_1',
       'BUREAU_BALANCE_2', 'BUREAU_BALANCE_3', 'BUREAU_BALANCE_4',
       'BUREAU_BALANCE_5', 'BUREAU_BALANCE_C', 'BUREAU_BALANCE_X',
       'BUREAU_BALANCE_MONTH_COUNT'],
      dtype='object')

In [126]:
bureau = bureau.merge(status_count, how='left', on='SK_ID_BUREAU')

In [127]:
bureau.describe()

,SK_ID_CURR,SK_ID_BUREAU,DAYS_CREDIT,CREDIT_DAY_OVERDUE,DAYS_CREDIT_ENDDATE,DAYS_ENDDATE_FACT,AMT_CREDIT_MAX_OVERDUE,CNT_CREDIT_PROLONG,AMT_CREDIT_SUM,AMT_CREDIT_SUM_DEBT,...,AMT_ANNUITY,BUREAU_BALANCE_0,BUREAU_BALANCE_1,BUREAU_BALANCE_2,BUREAU_BALANCE_3,BUREAU_BALANCE_4,BUREAU_BALANCE_5,BUREAU_BALANCE_C,BUREAU_BALANCE_X,BUREAU_BALANCE_MONTH_COUNT
count,1.716428e+06,1.716428e+06,1.716428e+06,1.716428e+06,1.610875e+06,1.082775e+06,5.919400e+05,1.716428e+06,1.716415e+06,1.458759e+06,...,4.896370e+05,774354.000000,774354.000000,774354.000000,774354.000000,774354.000000,774354.000000,774354.000000,774354.000000,774354.000000
mean,2.782149e+05,5.924434e+06,-1.142108e+03,8.181666e-01,5.105174e+02,-1.017437e+03,3.825418e+03,6.410406e-03,3.549946e+05,1.370851e+05,...,1.571276e+04,4.726299,0.014638,0.000723,0.000248,0.000161,0.001339,0.689338,2.205009,36.288669
std,1.029386e+05,5.322657e+05,7.951649e+02,3.654443e+01,4.994220e+03,7.140106e+02,2.060316e+05,9.622391e-02,1.149811e+06,6.774011e+05,...,3.258269e+05,65.733833,1.968870,0.008864,0.004092,0.003129,0.025614,18.667082,44.658598,25.449736
min,1.000010e+05,5.000000e+06,-2.922000e+03,0.000000e+00,-4.206000e+04,-4.202300e+04,0.000000e+00,0.000000e+00,0.000000e+00,-4.705600e+06,...,0.000000e+00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,1.888668e+05,5.463954e+06,-1.666000e+03,0.000000e+00,-1.138000e+03,-1.489000e+03,0.000000e+00,0.000000e+00,5.130000e+04,0.000000e+00,...,0.000000e+00,0.081631,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,15.000000
50%,2.780550e+05,5.926304e+06,-9.870000e+02,0.000000e+00,-3.300000e+02,-8.970000e+02,0.000000e+00,0.000000e+00,1.255185e+05,0.000000e+00,...,0.000000e+00,0.249984,0.000000,0.000000,0.000000,0.000000,0.000000,0.166657,0.020000,31.000000
75%,3.674260e+05,6.385681e+06,-4.740000e+02,0.000000e+00,4.740000e+02,-4.250000e+02,0.000000e+00,0.000000e+00,3.150000e+05,4.015350e+04,...,1.350000e+04,0.624961,0.000000,0.000000,0.000000,0.000000,0.000000,0.718728,0.249938,53.000000
max,4.562550e+05,6.843457e+06,0.000000e+00,2.792000e+03,3.119900e+04,0.000000e+00,1.159872e+08,9.000000e+00,5.850000e+08,1.701000e+08,...,1.184534e+08,1000.000000,1000.000000,0.833194,0.499875,0.666445,1.998002,1000.000000,1000.000000,96.000000


## Cleaning bureau data

In [128]:
# print number of unique item in each column
for col in bureau.columns:
    print('Unique item:', col, len(bureau[col].unique()))

Unique item: SK_ID_CURR 305811
Unique item: SK_ID_BUREAU 1716428
Unique item: CREDIT_ACTIVE 4
Unique item: CREDIT_CURRENCY 4
Unique item: DAYS_CREDIT 2923
Unique item: CREDIT_DAY_OVERDUE 942
Unique item: DAYS_CREDIT_ENDDATE 14097
Unique item: DAYS_ENDDATE_FACT 2918
Unique item: AMT_CREDIT_MAX_OVERDUE 68252
Unique item: CNT_CREDIT_PROLONG 10
Unique item: AMT_CREDIT_SUM 236709
Unique item: AMT_CREDIT_SUM_DEBT 226538
Unique item: AMT_CREDIT_SUM_LIMIT 51727
Unique item: AMT_CREDIT_SUM_OVERDUE 1616
Unique item: CREDIT_TYPE 15
Unique item: DAYS_CREDIT_UPDATE 2982
Unique item: AMT_ANNUITY 40322
Unique item: BUREAU_BALANCE_0 4638
Unique item: BUREAU_BALANCE_1 1834
Unique item: BUREAU_BALANCE_2 596
Unique item: BUREAU_BALANCE_3 347
Unique item: BUREAU_BALANCE_4 265
Unique item: BUREAU_BALANCE_5 1801
Unique item: BUREAU_BALANCE_C 4698
Unique item: BUREAU_BALANCE_X 4670
Unique item: BUREAU_BALANCE_MONTH_COUNT 98


In [129]:
# print number of missing value in each column
for col in bureau.columns:
    print('NA:', col, len(bureau.loc[bureau[col].isnull()]))

NA: SK_ID_CURR 0
NA: SK_ID_BUREAU 0
NA: CREDIT_ACTIVE 0
NA: CREDIT_CURRENCY 0
NA: DAYS_CREDIT 0
NA: CREDIT_DAY_OVERDUE 0
NA: DAYS_CREDIT_ENDDATE 105553
NA: DAYS_ENDDATE_FACT 633653
NA: AMT_CREDIT_MAX_OVERDUE 1124488
NA: CNT_CREDIT_PROLONG 0
NA: AMT_CREDIT_SUM 13
NA: AMT_CREDIT_SUM_DEBT 257669
NA: AMT_CREDIT_SUM_LIMIT 591780
NA: AMT_CREDIT_SUM_OVERDUE 0
NA: CREDIT_TYPE 0
NA: DAYS_CREDIT_UPDATE 0
NA: AMT_ANNUITY 1226791
NA: BUREAU_BALANCE_0 942074
NA: BUREAU_BALANCE_1 942074
NA: BUREAU_BALANCE_2 942074
NA: BUREAU_BALANCE_3 942074
NA: BUREAU_BALANCE_4 942074
NA: BUREAU_BALANCE_5 942074
NA: BUREAU_BALANCE_C 942074
NA: BUREAU_BALANCE_X 942074
NA: BUREAU_BALANCE_MONTH_COUNT 942074


In [130]:
# print number of 0 in each column
for col in bureau.columns:
    print('value=0:', col, len(bureau.loc[bureau[col] == 0]))

value=0: SK_ID_CURR 0
value=0: SK_ID_BUREAU 0
value=0: CREDIT_ACTIVE 0
value=0: CREDIT_CURRENCY 0
value=0: DAYS_CREDIT 25
value=0: CREDIT_DAY_OVERDUE 1712211
value=0: DAYS_CREDIT_ENDDATE 883
value=0: DAYS_ENDDATE_FACT 64
value=0: AMT_CREDIT_MAX_OVERDUE 470650
value=0: CNT_CREDIT_PROLONG 1707314
value=0: AMT_CREDIT_SUM 66582
value=0: AMT_CREDIT_SUM_DEBT 1016434
value=0: AMT_CREDIT_SUM_LIMIT 1050142
value=0: AMT_CREDIT_SUM_OVERDUE 1712270
value=0: CREDIT_TYPE 0
value=0: DAYS_CREDIT_UPDATE 605
value=0: AMT_ANNUITY 256915
value=0: BUREAU_BALANCE_0 111951
value=0: BUREAU_BALANCE_1 679211
value=0: BUREAU_BALANCE_2 761260
value=0: BUREAU_BALANCE_3 768467
value=0: BUREAU_BALANCE_4 770147
value=0: BUREAU_BALANCE_5 769913
value=0: BUREAU_BALANCE_C 356962
value=0: BUREAU_BALANCE_X 345823
value=0: BUREAU_BALANCE_MONTH_COUNT 5184


In [131]:
# def aggregate_do_nothing(bureau, col):
#     res = bureau[col]
#     return res.reindex(application['SK_ID_CURR']).fillna(0).rename('BUREAU_' + col)

In [132]:
def aggregate_count(bureau, col):
    # res = bureau[bureau['SK_ID_CURR'].isin(application['SK_ID_CURR'])]
    # res = res.groupby('SK_ID_CURR')[col].count()
    res = bureau.groupby('SK_ID_CURR')[col].count()
    return res.reindex(application['SK_ID_CURR']).fillna(0).rename('BUREAU_' + col + '_COUNT')

In [133]:
def aggregate_categorical(bureau, col):
    # res = bureau[bureau['SK_ID_CURR'].isin(application['SK_ID_CURR'])]
    # res = res.groupby('SK_ID_CURR')[col].value_counts().unstack()
    res = bureau.groupby('SK_ID_CURR')[col].value_counts().unstack()
    res = res.reindex(application['SK_ID_CURR']).fillna(0)
    res.columns = ['BUREAU_' + col + '_' + str(col_name) for col_name in res.columns]
    return res

In [134]:
def aggregate_avg_max_min_std_skw(bureau, col):
    # res = bureau[bureau['SK_ID_CURR'].isin(application['SK_ID_CURR'])]
    # res = res.groupby('SK_ID_CURR')[col].agg(['mean', 'max', 'min', 'std', 'skew'])
    res = bureau.groupby('SK_ID_CURR')[col].agg(['mean', 'max', 'min', 'std', 'skew'])
    res = res.reindex(application['SK_ID_CURR']).fillna(0)
    res.columns = ['BUREAU_' + col + '_' + col_name for col_name in res.columns]
    return res

In [135]:
def aggregate_avg_max_min_std_skw_nan_count(bureau, col):
    res = bureau[bureau['SK_ID_CURR'].isin(application['SK_ID_CURR'])]
    res = res.groupby('SK_ID_CURR')[col].agg(['mean', 'max', 'min', 'std', 'skew'])
    # count the number of nan
    res['nan_count'] = bureau.groupby('SK_ID_CURR')[col].apply(lambda x: x.isnull().sum())
    res = res.reindex(application['SK_ID_CURR']).fillna(0)
    res.columns = ['BUREAU_' + col + '_' + col_name for col_name in res.columns]
    return res

In [136]:
bureau_agg_dic = {'SK_ID_BUREAU': 'aggregate_count',
                  'CREDIT_ACTIVE': 'aggregate_categorical',
                  'CREDIT_CURRENCY': 'aggregate_categorical',
                  'DAYS_CREDIT': 'aggregate_avg_max_min_std_skw',
                  'CREDIT_DAY_OVERDUE': 'aggregate_avg_max_min_std_skw',
                  'DAYS_CREDIT_ENDDATE': 'aggregate_avg_max_min_std_skw_nan_count',
                  'DAYS_ENDDATE_FACT': 'aggregate_avg_max_min_std_skw_nan_count',
                  'AMT_CREDIT_MAX_OVERDUE': 'aggregate_avg_max_min_std_skw_nan_count',
                  'CNT_CREDIT_PROLONG': 'aggregate_avg_max_min_std_skw',
                  'AMT_CREDIT_SUM': 'aggregate_avg_max_min_std_skw',
                  'AMT_CREDIT_SUM_DEBT': 'aggregate_avg_max_min_std_skw_nan_count',
                  'AMT_CREDIT_SUM_LIMIT': 'aggregate_avg_max_min_std_skw_nan_count',
                  'AMT_CREDIT_SUM_OVERDUE': 'aggregate_avg_max_min_std_skw',
                  'DAYS_CREDIT_UPDATE': 'aggregate_avg_max_min_std_skw',
                  'AMT_ANNUITY': 'aggregate_avg_max_min_std_skw_nan_count',
                  'BUREAU_BALANCE_0': 'aggregate_avg_max_min_std_skw',
                  'BUREAU_BALANCE_1': 'aggregate_avg_max_min_std_skw',
                  'BUREAU_BALANCE_2': 'aggregate_avg_max_min_std_skw',
                  'BUREAU_BALANCE_3': 'aggregate_avg_max_min_std_skw',
                  'BUREAU_BALANCE_4': 'aggregate_avg_max_min_std_skw',
                  'BUREAU_BALANCE_5': 'aggregate_avg_max_min_std_skw',
                  'BUREAU_BALANCE_C': 'aggregate_avg_max_min_std_skw',
                  'BUREAU_BALANCE_X': 'aggregate_avg_max_min_std_skw',
                  'BUREAU_BALANCE_MONTH_COUNT': 'aggregate_avg_max_min_std_skw'}

In [137]:
home_credit_columns_description.where(home_credit_columns_description['Row'] == 'STATUS').dropna(axis=0, how='all')[
    'Description'].values[0]

'Status of Credit Bureau loan during the month (active, closed, DPD0-30,\x85 [C means closed, X means status unknown, 0 means no DPD, 1 means maximal did during month between 1-30, 2 means DPD 31-60,\x85 5 means DPD 120+ or sold or written off ] )'

In [138]:
# parallel processing
bureau_merging_result = Parallel(n_jobs=-1)(
    delayed(eval(bureau_agg_dic[col]))(bureau[bureau['SK_ID_CURR'].isin(application['SK_ID_CURR'])], col) for col in bureau_agg_dic.keys())# 6min

In [139]:
# merge all the result on SK_ID_CURR
bureau_merging_result = pd.concat(bureau_merging_result, axis=1)

In [147]:
bureau_merging_result.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 307511 entries, 100002 to 456255
Columns: 120 entries, BUREAU_SK_ID_BUREAU_COUNT to BUREAU_BUREAU_BALANCE_MONTH_COUNT_skew
dtypes: float64(120)
memory usage: 283.9 MB


In [148]:
application.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 307511 entries, 0 to 307510
Columns: 122 entries, SK_ID_CURR to AMT_REQ_CREDIT_BUREAU_YEAR
dtypes: float64(65), int64(41), object(16)
memory usage: 286.2+ MB


## Cleaning POS Cash data

In [141]:
POS_CASH_balance.describe()

,SK_ID_PREV,SK_ID_CURR,MONTHS_BALANCE,CNT_INSTALMENT,CNT_INSTALMENT_FUTURE,SK_DPD,SK_DPD_DEF
count,1.000136e+07,1.000136e+07,1.000136e+07,9.975287e+06,9.975271e+06,1.000136e+07,1.000136e+07
mean,1.903217e+06,2.784039e+05,-3.501259e+01,1.708965e+01,1.048384e+01,1.160693e+01,6.544684e-01
std,5.358465e+05,1.027637e+05,2.606657e+01,1.199506e+01,1.110906e+01,1.327140e+02,3.276249e+01
min,1.000001e+06,1.000010e+05,-9.600000e+01,1.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
25%,1.434405e+06,1.895500e+05,-5.400000e+01,1.000000e+01,3.000000e+00,0.000000e+00,0.000000e+00
50%,1.896565e+06,2.786540e+05,-2.800000e+01,1.200000e+01,7.000000e+00,0.000000e+00,0.000000e+00
75%,2.368963e+06,3.674290e+05,-1.300000e+01,2.400000e+01,1.400000e+01,0.000000e+00,0.000000e+00
max,2.843499e+06,4.562550e+05,-1.000000e+00,9.200000e+01,8.500000e+01,4.231000e+03,3.595000e+03


In [142]:
POS_CASH_balance

,SK_ID_PREV,SK_ID_CURR,MONTHS_BALANCE,CNT_INSTALMENT,CNT_INSTALMENT_FUTURE,NAME_CONTRACT_STATUS,SK_DPD,SK_DPD_DEF
0,1803195,182943,-31,48.0,45.0,Active,0,0
1,1715348,367990,-33,36.0,35.0,Active,0,0
2,1784872,397406,-32,12.0,9.0,Active,0,0
3,1903291,269225,-35,48.0,42.0,Active,0,0
4,2341044,334279,-35,36.0,35.0,Active,0,0
...,...,...,...,...,...,...,...,...
10001353,2448283,226558,-20,6.0,0.0,Active,843,0
10001354,1717234,141565,-19,12.0,0.0,Active,602,0
10001355,1283126,315695,-21,10.0,0.0,Active,609,0
10001356,1082516,450255,-22,12.0,0.0,Active,614,0


In [143]:
[item for item in application['SK_ID_CURR'] if item not in POS_CASH_balance['SK_ID_CURR']]

[]

In [155]:
POS_CASH_balance['SK_ID_CURR'].unique()

array([182943, 367990, 397406, ..., 394310, 240059, 144536], dtype=int64)

In [156]:
POS_CASH_balance['SK_ID_PREV'].unique()

array([1803195, 1715348, 1784872, ..., 2452932, 1980876, 2308284],
      dtype=int64)

In [159]:
print(POS_CASH_balance['SK_ID_PREV'])

0           1803195
1           1715348
2           1784872
3           1903291
4           2341044
             ...   
10001353    2448283
10001354    1717234
10001355    1283126
10001356    1082516
10001357    1259607
Name: SK_ID_PREV, Length: 10001358, dtype: int64


In [158]:
POS_CASH_balance['SK_ID_CURR'].isna().sum()

0

In [153]:
POS_CASH_balance[POS_CASH_balance['SK_ID_CURR'].isin(application['SK_ID_CURR'])]['SK_ID_CURR'].unique()

array([182943, 367990, 397406, ..., 394310, 240059, 144536], dtype=int64)

In [154]:
credit_card_balance[credit_card_balance['SK_ID_CURR'].isin(application['SK_ID_CURR'])]['SK_ID_CURR'].unique()

array([363914, 337855, 126868, ..., 174455, 315041, 382749], dtype=int64)

In [ ]:
previous_application

In [150]:
{prev_id:now_id for (prev_id,now_id) in zip(POS_CASH_balance['SK_ID_PREV'],POS_CASH_balance['SK_ID_CURR']) if prev_id in credit_card_balance['SK_ID_PREV'] and now_id in credit_card_balance['SK_ID_CURR']}

KeyboardInterrupt: 

## Cleaning Credit card data

In [145]:
credit_card_balance

,SK_ID_PREV,SK_ID_CURR,MONTHS_BALANCE,AMT_BALANCE,AMT_CREDIT_LIMIT_ACTUAL,AMT_DRAWINGS_ATM_CURRENT,AMT_DRAWINGS_CURRENT,AMT_DRAWINGS_OTHER_CURRENT,AMT_DRAWINGS_POS_CURRENT,AMT_INST_MIN_REGULARITY,...,AMT_RECIVABLE,AMT_TOTAL_RECEIVABLE,CNT_DRAWINGS_ATM_CURRENT,CNT_DRAWINGS_CURRENT,CNT_DRAWINGS_OTHER_CURRENT,CNT_DRAWINGS_POS_CURRENT,CNT_INSTALMENT_MATURE_CUM,NAME_CONTRACT_STATUS,SK_DPD,SK_DPD_DEF
0,2562384,378907,-6,56.970,135000,0.0,877.5,0.0,877.5,1700.325,...,0.000,0.000,0.0,1,0.0,1.0,35.0,Active,0,0
1,2582071,363914,-1,63975.555,45000,2250.0,2250.0,0.0,0.0,2250.000,...,64875.555,64875.555,1.0,1,0.0,0.0,69.0,Active,0,0
2,1740877,371185,-7,31815.225,450000,0.0,0.0,0.0,0.0,2250.000,...,31460.085,31460.085,0.0,0,0.0,0.0,30.0,Active,0,0
3,1389973,337855,-4,236572.110,225000,2250.0,2250.0,0.0,0.0,11795.760,...,233048.970,233048.970,1.0,1,0.0,0.0,10.0,Active,0,0
4,1891521,126868,-1,453919.455,450000,0.0,11547.0,0.0,11547.0,22924.890,...,453919.455,453919.455,0.0,1,0.0,1.0,101.0,Active,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3840307,1036507,328243,-9,0.000,45000,NaN,0.0,NaN,NaN,0.000,...,0.000,0.000,NaN,0,NaN,NaN,0.0,Active,0,0
3840308,1714892,347207,-9,0.000,45000,0.0,0.0,0.0,0.0,0.000,...,0.000,0.000,0.0,0,0.0,0.0,23.0,Active,0,0
3840309,1302323,215757,-9,275784.975,585000,270000.0,270000.0,0.0,0.0,2250.000,...,273093.975,273093.975,2.0,2,0.0,0.0,18.0,Active,0,0
3840310,1624872,430337,-10,0.000,450000,NaN,0.0,NaN,NaN,0.000,...,0.000,0.000,NaN,0,NaN,NaN,0.0,Active,0,0


In [146]:
[item for item in credit_card_balance['SK_ID_CURR'] if item not in application.columns]

[378907,
 363914,
 371185,
 337855,
 126868,
 380010,
 171320,
 118650,
 367360,
 203885,
 209660,
 340339,
 302517,
 171537,
 303581,
 289902,
 255981,
 276642,
 302450,
 231608,
 259807,
 217113,
 132481,
 105755,
 323730,
 195164,
 184131,
 387286,
 213409,
 406607,
 427122,
 388056,
 337802,
 193247,
 223794,
 234016,
 379250,
 441030,
 384410,
 330465,
 215299,
 147323,
 425246,
 313797,
 352647,
 399970,
 116448,
 121258,
 104946,
 104761,
 271657,
 352319,
 215709,
 113120,
 367562,
 166121,
 343837,
 217747,
 224252,
 128242,
 162464,
 343308,
 249286,
 252742,
 118055,
 277880,
 166894,
 293607,
 336961,
 140209,
 114707,
 340224,
 277061,
 311117,
 103029,
 214797,
 271670,
 294744,
 152835,
 183994,
 385344,
 439525,
 341787,
 263564,
 353300,
 274118,
 216234,
 224220,
 226681,
 186806,
 436212,
 274239,
 161019,
 302820,
 416815,
 375482,
 354226,
 285273,
 283851,
 281671,
 318823,
 443277,
 218365,
 148430,
 155018,
 317323,
 144297,
 440902,
 415488,
 263672,
 111058,
 